In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions._


spark: SparkSession = org.apache.spark.sql.SparkSession@37d22f14

In [2]:
import spark.implicits._

case class Seat(id: Int, student: String)

val seat: Seq[(Int, String)] = Seq(
  (1, "Abbot"),
  (2, "Doris"),
  (3, "Emerson"),
  (4, "Green"),
  (5, "Jeames")
)

val seatDS = seat.toDF("id", "student").as[Seat]


import spark.implicits._


defined class Seat
seat: Seq[(Int, String)] = List(
  (1, "Abbot"),
  (2, "Doris"),
  (3, "Emerson"),
  (4, "Green"),
  (5, "Jeames")
)
seatDS: Dataset[Seat] = [id: int, student: string]

Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.

Return the result table ordered by id in ascending order.

The result format is in the following example.

| id | student |
|----|---------|
| 1  | Doris   |
| 2  | Abbot   |
| 3  | Green   |
| 4  | Emerson |
| 5  | Jeames  |

In [3]:
val maxId: Int = seatDS.agg(max("id")).collect()(0)(0).asInstanceOf[Int]
seatDS.withColumn("lag", lag(col("student"), 1).over(Window.orderBy("id")))
    .withColumn("lead", lead(col("student"), 1).over(Window.orderBy("id")))
    .select(
        col("id"),
        when((col("id") % 2 === 1) && (col("id") !== lit(5)), col("lead"))
        .when(col("id") % 2 === 0, col("lag"))
        .otherwise(col("student"))
        .as("student")
    )
    .show()

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



maxId: Int = 5

In [4]:
spark.stop()